<a href="https://colab.research.google.com/github/miguelangel17z/PDS/blob/main/Pr%C3%A1ctica_3_Est_Datos_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica tres: Cálculo de Integrales por los Métodos de Simpson en Paralelo usando OpenMP**


### **Profesor:** Carlos Álvarez Henao.

### **Integrantes:**

*   Miguel Ángel Garcia Osorio.
*   Dorian Alejandro Guisao Ospina.

### **Índice:**
- [Simpson 1/3 Aplicación simple](#simpson-13-aplicacion-simple)
- [Simpson 1/3 Aplicación Compuesta](#simpson-13-aplicacion-compuesta)
- [Simpson 3/8 Simple](#simpson-38-simple)
- [Simpson 3/8 Multiple](#simpson-38-multiple)





#### **Nota:**  Se utilizó el entorno MSYS2 junto con el compilador MinGW-w64 para compilar los archivos escritos en C, y poder generar los ejecutables.

####Las pruebas se llevaron en dos computadoras distinta:

####**1/3 simple y multiple:**

####**3/8 simple y multiple:** acer nitro 5 con un procesador 12th Gen Intel(R) Core(TM) i5-12500H con 12 nucleos y 16 hilos.



# ***Método de Simpson 1/3.***

## **Simpson 1/3  Aplicación Simple**

La aplicación simple del método de Simpson 1/3 Simple consiste en evaluar la siguiente fórmula:

$$
\int_{a}^{b} f(x) \, dx \approx \frac{1}{3} \cdot h \cdot \left(f(a) + 4f(a + h) + f(b)\right)
$$

donde:

- $( h = \frac{b - a}{n} )$
- $( n = 2 )$

Aplicando esto a nuestro ejercicio, con $f(x) = e^{-x^2}$, $a = -1$ y $b = 1$, tenemos:

$$h = \frac{1 - (-1)}{2} = \frac{2}{2} = 1$$

Entonces, evaluamos la fórmula:

$$
\int_{-1}^{1} e^{-x^2} \, dx \approx \frac{1}{3} \cdot 1 \cdot \left( f(-1) + 4f(0) + f(1) \right)
$$

Calculamos los valores:

- $f(-1) = e^{-1} \approx 0.367879$
- $f(0) = e^{0} = 1$
- $f(1) = e^{-1} \approx 0.367879$

Sustituimos:

$$
\int_{-1}^{1} e^{-x^2} \, dx \approx \frac{1}{3} \cdot \left( 0.367879 + 4(1) + 0.367879 \right)
$$

$$
\int_{-1}^{1} e^{-x^2} \, dx \approx \frac{1}{3} \cdot 4.735758 \approx 1.578586
$$

Por lo tanto, el resultado aproximado utilizando la **regla de Simpson 1/3 simple** es:

$$
\boxed{1.578586}
$$

Ahora, veamos esta implementacion en el lenguaje de programación C:


**Implementación en** ***C*** **:**

```c
#include <stdio.h>
#include <math.h> // Para poder definir nuestra función

// Definimos nuestra función: e^[-x^2]
double f(double x) {
    return exp(-x * x);
}

// Función para aplicar la fórmula del método de Simpson 1/3 simple
double simpson_simple(double a, double b) {

    int n = 2; // Como se dijo anteriormente aqui 'n' es siempre 2.
    double h = (b - a) / n; // Calculamos el ancho del subintervalos central.
    double a_sum_h = a + h;

    // Aplicamos la fórmula de Simpson 1/3 simple para calcular en valor de aprox de I.
    double resul_I = (1.0 / 3.0) * h * (f(a) + 4 * f(a_sum_h) + f(b));  
    return resul_I;
}

// Función principal
int main() {
    double a = -1.0, b = 1.0; // Definimos los limites de integración

    // Calculamos el valor aprox de I con el método de Simpson Simple.
    double r_simp_simple = simpson_simple(a, b);
    
    // Imprimimos el resultado de I.
    printf("Resultado Simpson simple = %.15f\n", r_simp_simple);
    return 0;
}
```
**Salida del código:**

![Output_Simpson_Simple](https://raw.githubusercontent.com/DorianAlejandroGuisaoO/Practica3_Struc_Datos/refs/heads/main/imagenes_P3/Salida%20Simpson%201-3%20Simple.png)

**Finalmente, esta versión del método de Simpson 1/3 no es paralelizable, ya que, como se vio en clase, el principal objetivo de la paralelización son los ciclos, y en este caso no hay ninguno. Solo se realizan operaciones secuenciales, como asignaciones y evaluaciones de funciones, que no permiten aprovechar técnicas de paralelización. Por lo tanto, esta implementación es completamente secuencial.**

## **Simpson 1/3  Aplicación Compuesta**

El método de Simpson 1/3 compuesto, a diferencia del simple si nos da un valor muy aproximado de la integral que queramos integrar (debe ser continua), la idea principal es aplicar la regla de Simpson 1/3 (simple) repetidamente en cada par de subintervalos, lo que mejora la precisión en comparación con una única aplicación.

El intervalo total se divide en $n$ subintervalos, donde $n$ debe ser un número par. La longitud de cada subintervalo ($h$) es:



$$
h = \frac{b - a}{n}
$$

Donde: $a$ y $b$ son los límites inferior y superior de integración, respectivamente. Ahora veamos la fórmula del método de Simpson 1/3 compuesto.

* **Fórmula Simpson Compuesto:**

$$I = \int_{a}^{b} f(x) \, dx \ \approx  \  \frac{h}{3} \left[ f(a)\ +\  4 \sum_{i=1, \text{impar}}^{n-1} f(x_i)\ + \ 2 \sum_{i=2, \text{par}}^{n-2} f(x_i)\ +\ f(b) \right]$$

donde:

- $x_0 = a$, $x_n = b$

- $x_i = a + i \cdot h$ es nuestro $i_{esimo}$ punto de evaluación dentro del intervalo.

Aplicandola a nuestro ejercicio seria:

$$I = \int_{-1}^{1} e^{-x^2} \, dx \ \approx  \  \frac{h}{3} \left[ f(-1)\ +\  4 \sum_{i=1, \text{impar}}^{n-1} f(x_i)\ + \ 2 \sum_{i=2, \text{par}}^{n-2} f(x_i)\ +\ f(1) \right]$$

Finalmente, veamos las implementaciones secuencial y paralelizada en el lenguaje de programación C.






### **Simpson 1/3 Compuesto: Implementación** ***Secuencial.***

La implementación secuencial del método de Simpson 1/3 Compuesto se desarrolla de la siguiente manera:

**1.** Definir nuestra función: $f(x) = e^{-x^2}$

$$$$
**2.** Crear una función para calcular el valor de $I$; la función recibirá como parámetros $a$, $b$ y $n$.

  2.1. Ya que $h$ depende de $n$, calculamos el valor de $h$ dentro de la función:

  2.2. Luego de tener $h$, podemos calcular las sumatorias (con ciclos `for`) de todos los términos de los subintervalos pares e impares.

  2.3. Una vez tengamos estas dos cosas, ya podemos evaluar la fórmula de Simpson 1/3 Compuesto.
$$$$
**3.** Función *main*, donde se inicializan $a$, $b$ y $n$, para calcular el valor aproximado de $I$.

```c
#include <stdio.h>
#include <math.h>
#include <sys/time.h>  

// Definimos nuestra función: e^[-x^2]
double f(double x) {
    return exp(-x * x);
}

// Código secuencial para el método de Simpson 1/3.
double simpson_secuen(double a, double b, int n) {
    // Calculamos el ancho de cada subintervalo.
    double h = (b - a) / n;

    // Inicializamos las sumas de los subintervalos pares e impares.
    double sum_pares = 0.0, sum_impares = 0.0;
    
    // Calculamos la suma de todos los términos de los subintervalos pares.
     for (int i = 2; i <= n - 2; i += 2) {
        sum_pares += f(a + i * h);
    }

    // Calculamos la suma de todos los términos de los subintervalos impares.
    for (int i = 1; i <= n - 1; i += 2) {
        sum_impares += f(a + i * h);
    }

    // Aplicamos la fórmula de Simpson 1/3 y calculamos el valor aproximado de I.
    double resultado_I = h / 3 * (f(a) + 4 * sum_impares + 2 * sum_pares + f(b));
    return resultado_I;
}

// Función para obtener el tiempo en segundos usando gettimeofday
double get_time() {
    struct timeval t;
    gettimeofday(&t, NULL);
    return t.tv_sec + t.tv_usec / 1000000.0;  // Retorna el tiempo en segundos con precisión de microsegundos
}

// Función principal.
int main() {
    // Definimos los límites inferior (a) y superior (b).
    double a = -1.0, b = 1.0;
    int n = 2000000000; // Definimos 'n' subintervalos.

    // Medimos el tiempo de ejecución secuencialmente
    double start1 = get_time();
    double resul_secuen = simpson_secuen(a, b, n);
    double end1 = get_time();
    double tiempo_secuencial = end1 - start1;

    // Imprimimos los resultados.
    printf("Número de subintervalos (n) = %lld\n", n);
    printf("Resultado secuencial: %f\n", resul_secuen);
    printf("Tiempo secuencial: %f segundos\n", tiempo_secuencial);
}
```

* **Resultado de aplicar el método de Simpson 1/3 Compuesto con  $n = 2,000,000,000 $**

![Output_Simpson_Compuesto_Secuencial](https://raw.githubusercontent.com/DorianAlejandroGuisaoO/Practica3_Struc_Datos/refs/heads/main/imagenes_P3/T1_Simpson_1-3_Comp_Secuencial.png)



### **Simpson 1/3 Compuesto: Implementación** ***Paralelizada.***

Tomando como punto de partida la implementación anterior podemos ver que hay dos partes del codigo que son buenos candidatos para la parelizacion:

1. ```c
    // Calculamos la suma de todos los términos de los subintervalos pares.
     for (int i = 2; i <= n - 2; i += 2) {
        sum_pares += f(a + i * h);
    }
    ```

2. ```c
    // Calculamos la suma de todos los términos de los subintervalos impares.
    for (int i = 1; i <= n - 1; i += 2) {
        sum_impares += f(a + i * h);
    }
    ```


Estas dos partes son ideales para paralelizar ya que son bucles `for`. Es decir, nuestro objetivo principal para la parelelización. Para parelelizar estos ciclos usaremos OpenMP.

* **Paralelización:** Para paralelizar ambos ciclos utilizamos la directiva de compilación `#pragma omp parallel for`, seguida de las cláusulas `reduction(+:sum_pares)` y `reduction(+:sum_impares)` respectivamente para cada ciclo. Estas cláusulas se emplean para evitar condiciones de carrera entre los hilos al calcular las sumatorias.

  Finalmente, el ciclo que calcula la sumatoria de los términos correspondientes a los subintervalos pares quedaría así:

  ```c
  // Calculamos la suma de todos los términos de los subintervalos pares.
  #pragma omp parallel for reduction(+:sum_pares)
    for (int i = 2; i <= n - 2; i += 2) {
        sum_pares += f(a + i * h);
    }
  ```


* **Implentación en C de la versión paralelizada de Simpson 1/3 Compuesto:**

  ```c
  #include <omp.h>
  #include <stdio.h>
  #include <math.h>
  #include <sys/time.h>  

  // Definimos nuestra función: e^[-x^2]
  double f(double x) {
    return exp(-x * x);
  }

  // Paralelizamos el código secuencial para el método de Simpson 1/3 simple.
  double simpson_paralelo(double a, double b, int n) {
    // Calculamos el ancho de cada subintervalo.
    double h = (b - a) / n;

    // Inicializamos las sumas de los subintervalos pares e impares.
    double sum_pares = 0.0, sum_impares = 0.0;
    
    // Paralelizamos el cálculo de todos los términos de los subintervalos pares.
    #pragma omp parallel for reduction(+:sum_pares)
    for (int i = 2; i <= n - 2; i += 2) {
        sum_pares += f(a + i * h);
    }

    // Paralelizamos el cálculo de todos los términos de los subintervalos impares.
    #pragma omp parallel for reduction(+:sum_impares)
    for (int i = 1; i <= n - 1; i += 2) {
        sum_impares += f(a + i * h);
    }

    // Aplicamos la fórmula de Simpson 1/3 y calculamos el valor aproximado de I.
    double resultado_I = h / 3 * (f(a) + 4 * sum_impares + 2 * sum_pares + f(b));
    return resultado_I;
  }

  // Función para obtener el tiempo en segundos usando gettimeofday
  double get_time() {
    struct timeval t;
    gettimeofday(&t, NULL);
    return t.tv_sec + t.tv_usec / 1000000.0;  // Retorna el tiempo en segundos con precisión de microsegundos
  }

  // Función principal.
  int main() {
    // Definimos los límites inferior (a) y superior (b).
    double a = -1.0, b = 1.0;
    int n = 2000000000; // Definimos 'n' subintervalos.

    // Establecemos el número de hilos a usar para el cálculo en paralelo.     
    omp_set_num_threads(12);

    // Medimos el tiempo de ejecución con paralelización
    double start2 = get_time();
    double resul_paralelo = simpson_paralelo(a, b, n);
    double end2 = get_time();
    double tiempo_paralelo = end2 - start2;

    // Imprimimos los resultados.
    printf("Número de subintervalos (n) = %lld\n", n);
    printf("Resultado paralelo:   %f\n", resul_paralelo);
    printf("Tiempo paralelo:   %f segundos\n", tiempo_paralelo);
}

  ```

* ****Resultado de aplicar el método de Simpson 1/3 Compuesto paralelizado con  $n = 2,000,000,000$****

  ![Output_Simpson_Compuesto_Paralelizado](https://raw.githubusercontent.com/DorianAlejandroGuisaoO/Practica3_Struc_Datos/refs/heads/main/imagenes_P3/T1_Simpson_1-3_Comp_Paralelo.jpg.png)

### **Conclusiones Simpson 1/3 Compuesto** ***(Speed-Up).***

Como se puede ver la implementación paralelizada es casi muchas veces más rápida que la secuencial, ahora calculemos el ***Speed-Up*** $(S)$.


$$S = \frac{T_{\text{sec}}}{T_{\text{par}}}$$

Con los datos que obtuvimos anteriormente el ***Speed-up*** fue de:

$$S = \frac{99.504411}{12.335698}  =  8.06678$$


Esto quiere decir que nuestro código paralelo ***8.06678*** veces más rápido que el código secuencial.


Tambien podemos comprobar que el método de Simpson 1/3 Compuesto nos dio el valor exacto de $I$:

[Resultado][https://www.wolframalpha.com/input?i2d=true&i=Integrate%5BExp%5B-x*x%5D%2C%7Bx%2C-1%2C1%7D%5D&lang=es]



##**Simpson 3/8 simple**

Formula:

$$
\int_a^b f(x)\, dx \approx \frac{3h}{8} \left[ f(a) + 3f(a+h) + 3f(a+2h) + f(b) \right]$$

donde

$$ h = \frac{(b-a)}{3}$$

Este método se encarga de aproximar el valor de la integral dividiendo el intervalo en 3 subintervalos de la misma longitud. Para esto, claramente tendremos que tener un número de intervalos múltiplo de 3.

#### **Implementacion en c secuencial:**
La implementación es bastante sencilla, y solo trata de replicar la formula en c.

```c
#include <stdio.h>
#include <math.h>
#include <time.h>

double f(double x){ // funcion a resolver e^(-x*x)
    return exp(-x * x);
}

double simpson_series_simple(double a, double b){

    double h = (b-a)/3; // definimos los sub intervalos

    double form_part_one = (3*h)/8;  // Primera parte de la formula

    double form_part_two = f(a) + 3 * f(a + h) + 3 * f(a + 2*h) + f(b); // segunda parte de la formula

    return form_part_one*form_part_two;
}

int main(int argc, char const *argv[]) {

    // medimos el tiempo
    double start_time = clock();

    double resultado = simpson_series_simple(-1,1);

    double end_time = clock();

    double time = (double)(end_time - start_time)/ CLOCKS_PER_SEC;

    // output
    printf("Resultado integral de forma secuencial: %f",resultado);
    printf("\nTiempo secuencial: %.20f",time);

    return 0;
}


```
##### **OUTPUT:**
![output de metodo simple secuencial](https://github.com/DorianAlejandroGuisaoO/Practica3_Struc_Datos/blob/main/imagenes_P3/output_3_8_simple_sec.png?raw=true)

Aqui primero compilamos para crear el archivo ejecutable y luego simplemente ejecutamos ese archivo. Debido a que la funcion `clock()` no es tan precisa nos devuelve un resultado de 0 segundos.

###**Implementacion en c paralelizada con OpenMp**:

En este caso, se tuvo que modificar un poco el código para poder paralelizarlo. Ahora, resolvemos cada función por separado y luego juntamos los resultados, en lugar de hacerlo todo en una misma línea. Esto nos permite realizar una paralelización utilizando el método sections. Por otro lado,  usamos la mayor cantidad de hilos posibles que permite el procesador.

```c
#include <stdio.h>  
#include <math.h>
#include <omp.h>

double f(double x){ // funcion a resolver e^(-x*x)
    return exp(-x * x);
}


double simpson_series_simple_paralelizado(double a, double b){

    double h = (b-a)/3; // definimos los sub intervalos
    double form_part_one = (3*h)/8;  // Primera parte de la formula
    
    double fx1,fx2,fx3,fx4; // separamos cada funcion para poder paralelizar

    # pragma omp parallel sections // hacemos que cada hilo resuelva una funcion
    {

        #pragma omp section
        fx1 = f(a);

        #pragma omp section
        fx2 = f(a + h);

        #pragma omp section
        fx3 = f(a + (2*h));

        #pragma omp section
        fx4 = f(b);
    }

    return form_part_one*(fx1 + 3*fx2 + 3*fx3 + fx4); // formula completa
    }


int main(int argc, char const *argv[]) {
    omp_set_num_threads(16); // definimos el numero de hilos

    // tomamos el tiempo
    double start_time = omp_get_wtime();  // tiempo de inicio

    double resultado = simpson_series_simple_paralelizado(-1,1);

    double end_time = omp_get_wtime();  // tiempo final

    printf("Resultado integral con paralelizacion: %f",resultado);
    printf("\nTiempo con paralelizacion: %.20f",end_time-start_time);


   

    return 0;
}


```

##### **OUTPUT:**
![output de metodo simple secuencial](https://github.com/DorianAlejandroGuisaoO/Practica3_Struc_Datos/blob/main/imagenes_P3/output_3_8_simple_parl.png?raw=true)

En este caso la funcion `omp_get_wtime()` es mucho mas precisa y puede devolvernos un valor real.

### **SPEED-UP:**
Debido a que ambos valores son muy cercanos a 0 y en el caso de uno de ellos, no es posible saber su valor exacto debido a limitaciones con el temporizador, el speed-up no es para nada significativo y en realidad nos ayuda concluir que debido a la simplicidad del código, la paralelización resulta un poco innecesaria, ya que de manera secuencial funciona correctamente sin necesidad de paralelizarse.








## **Simpson 3/8 multiple**

Formula:

$$
\int_a^b f(x)\,dx \approx \frac{3h}{8} \sum_{j=1}^{n/3} \left[ f(x_{3j - 3}) + 3f(x_{3j - 2}) + 3f(x_{3j - 1}) + f(x_{3j}) \right] $$

donde

$$ h = \frac{b - a}{n} $$

y

$$n \bmod 3 = 0$$

Esta es una técnica para aproximar el valor de una integral. dividiendo el intervalo `[a,b]` en subintervalos iguales, asegurándose de que el número de subintervalos sea múltiplo de 3. Luego, toma ciertos puntos dentro de esos subintervalos y les asigna pesos especificos, para despues unirlos mediante una sumatoria.

### **Implementacion en c secuencial:**

En este caso tuvimos que manipular de forma algebraica la formula original, distribuyendo la sumatoria en cada uno de los terminos, quedandonos la fomula de esta manera:

$$ \int_a^b f(x)\,dx \approx \frac{3h}{8} \left( \sum_{j=1}^{n/3} f(x_{3j - 3}) + 3 \sum_{j=1}^{n/3} f(x_{3j - 2}) + 3 \sum_{j=1}^{n/3} f(x_{3j - 1}) + \sum_{j=1}^{n/3} f(x_{3j}) \right) $$

Ya con esto la implementacion se vuelve bastante sencilla donde cada sumatoria es un ciclo que va sumando el valor que va dando la función mientras `j` aumenta.

Usamos un n bastante alto para asi poder percibir mejor las diferencias de tiempo entre el metodo secuencial y paralelizado.

```c
#include <stdio.h>
#include <math.h>
#include <time.h>

double f(double x){ // funcion a resolver e^(-x*x)
    return exp(-x * x);
}

double simpson_series_multiple(double a, double b, int n){

    if((n % 3) != 0){ // verificamos que n si sea multiplo 3
        return 0;
    }else{ // si cumple resolvemos la integral

        double sum1 = 0.0, sum2 = 0.0,sum3 = 0.0, sum4 = 0.0; // definimos la variable donde se guardara la suma de cada funcion

        double h = (b-a)/n;  

        // hacemos cada sumatoria por separado
        for(int j = 1; j<=n/3; j++){ // primer termino
            sum1 += f(a + (3*j - 3)*h);
        }

        for(int j = 1; j<=n/3; j++){ // segundo termino
            sum2 += f(a + (3*j - 2)*h);
        }

        for(int j = 1; j<=n/3; j++){ // tercer termino
            sum3 += f(a + (3*j - 1)*h);
        }

        for(int j = 1; j<=n/3; j++){ // cuarto termino
            sum4 += f(a + (3*j)*h);
        }

        double integral = (3*h/8)*(sum1 + 3*sum2 + 3*sum3 + sum4);

        return integral;

    }
}

int main(int argc, char const *argv[])
{
    // medimos el tiempo
    double start_time = clock();

    double resultado = simpson_series_multiple(-1,1,300000000);

    double end_time = clock();

    double time = (double)(end_time - start_time)/ CLOCKS_PER_SEC;
    // output
    printf("Integral por metodo de simpson 3/8 multiple: %.10f",resultado);
    printf("\nTiempo secuencial: %.10f",time);
    return 0;
}

```
##### **OUTPUT:**

**donde `n = 300,000,000`**
![ouput secuencia multiple 3/8](https://github.com/DorianAlejandroGuisaoO/Practica3_Struc_Datos/blob/main/imagenes_P3/output_3_8_mult_sec.png?raw=true)

Aca se puede notar claramente como logramos obtener un valor mucho mas acertado, sin embargo, el tiempo es bastante alto.

### **Implementacion en c paralelizado:**
En este caso, la paralelización fue bastante sencilla, sin necesidad de cambiar el código, ya que está separada en ciclos `for`. Esto nos permite hacer una paralelización utilizando `omp for reduction`. Al igual que en el caso anterior, usamos la mayor cantidad de hilos posibles que permite el procesador.

```c
#include <stdio.h>
#include <math.h>
#include <time.h>
#include <omp.h>

double f(double x){ // funcion a resolver e^(-x*x)
    return exp(-x * x);
}

double simpson_series_multiple(double a, double b, int n){

    if((n % 3) != 0){ // verificamos que n si sea multiplo 3
        return 0;
    }else{ // si cumple resolvemos la integral

        double sum1 = 0.0, sum2 = 0.0,sum3 = 0.0, sum4 = 0.0; // definimos la variable donde se guardara la suma de cada funcion

        double h = (b-a)/n;  

        // hacemos cada sumatoria por separado
        #pragma omp parallel
        {
        #pragma omp for reduction(+:sum1)
        for(int j = 1; j<=n/3; j++){ // primer termino
            sum1 += f(a + (3*j - 3)*h);
        }

        #pragma omp for reduction(+:sum2)
        for(int j = 1; j<=n/3; j++){ // segundo termino
            sum2 += f(a + (3*j - 2)*h);
        }

        #pragma omp for reduction(+:sum3)
        for(int j = 1; j<=n/3; j++){ // tercer termino
            sum3 += f(a + (3*j - 1)*h);
        }

        #pragma omp for reduction(+:sum4)
        for(int j = 1; j<=n/3; j++){ // cuarto termino
            sum4 += f(a + (3*j)*h);
        }
    }
        double integral = (3*h/8)*(sum1 + 3*sum2 + 3*sum3 + sum4);

        return integral;

    }
}

int main(int argc, char const *argv[])
{
    omp_set_num_threads(16); // usamos la mayor cantidad de hilos posibles en este pc
    // medimos el tiempo
    double start_time = omp_get_wtime();

    double resultado = simpson_series_multiple(-1,1,300000000);

    double end_time = omp_get_wtime();;

    //output
    printf("Integral por metodo de simpson 3/8 multiple: %.10f",resultado);
    printf("\nTiempo secuencial: %.10f",end_time-start_time);
    return 0;
}

```

#### **OUPUT:**
![output mult parl 3/8](https://github.com/DorianAlejandroGuisaoO/Practica3_Struc_Datos/blob/main/imagenes_P3/output_3_8_mult_parl.png?raw=true)

En este caso vemos que el tiempo es mucho menor que en su versión secuencial, demostrandonos claramente que este es un codigo que necesitaba ser paralelizado.

#### **SPEED-UP(S):**

Tenemos que:
$$S = \frac{T_{\text{sec}}}{T_{\text{par}}}$$

Con los datos que obtuvimos anteriormente el ***Speed-up*** fue de:

$$S = \frac{19.6500000000}{2.9479999542}  =  6.66553606$$

Esto nos dice que el codigo con paralelización fue aproximadamente 6.7 veces mas rapido que en su version secuencial, lo que de nuevo nos demuestra la importancia de aplicar la paralelizacion en estos casos donde se le es posible.








